In [ ]:
from dotenv import load_dotenv
import os
# Get the path to the root directory of the repository
root_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))

# Load environment variables from .env file in the root directory
load_dotenv(os.path.join(root_dir, '.env'))

In [ ]:
from langsmith import Client

experiment_name = "Hotel missconf fix error double export of RCA worker"

langsmith_client = Client()
runs = langsmith_client.list_runs(
    project_name=os.environ.get("LANGSMITH_PROJECT"),
    filter=f'eq(name, "{experiment_name}")',
    limit=1
)

run = next(iter(runs), None)

In [ ]:
if run:
    print(f"Run ID: {run.id}")
    print(f"Run Name: {run.name}")
    print(f"Status: {run.status}")
    print(f"Total Tokens: {run.total_tokens}")
    print(f"Start Time: {run.start_time}")
    print(f"End Time: {run.end_time}")
    print(f"Duration: {(run.end_time - run.start_time).total_seconds()}s")

In [ ]:
if run:
    print(f"Total Run Tokens: {run.total_tokens}\n")
    
    print("📊 Token Usage by Agent:")
    print("-" * 50)
    
    # Get all child runs
    child_runs = list(langsmith_client.list_runs(parent_run_id=run.id))
    
    for agent_run in child_runs:
        print(f"\n🔹 {agent_run.name}")
        print(f"   Total Tokens: {agent_run.total_tokens}")
        print(f"   Status: {agent_run.status}")
        print(f"   Cost: ${agent_run.completion_cost:.6f}" if agent_run.completion_cost else "   Cost: N/A")

In [ ]:
if run:
    print(f"Total Run Tokens: {run.total_tokens}\n")
    
    print("📊 Token Usage by Agent:")
    print("-" * 50)
    
    # Get all child runs
    child_runs = list(langsmith_client.list_runs(parent_run_id=run.id))
    
    # Aggregate token usage by agent name
    agent_stats = {}
    
    for agent_run in child_runs:
        agent_name = agent_run.name
        
        if agent_name not in agent_stats:
            agent_stats[agent_name] = {
                "total_tokens": 0,
                "input_tokens": 0,
                "output_tokens": 0,
                "cost": 0.0,
                "runs_count": 0
            }
        
        agent_stats[agent_name]["total_tokens"] += agent_run.total_tokens or 0
        agent_stats[agent_name]["input_tokens"] += (agent_run.input_tokens or 0)
        agent_stats[agent_name]["output_tokens"] += (agent_run.output_tokens or 0)
        agent_stats[agent_name]["cost"] += float((agent_run.completion_cost or 0.0))
        agent_stats[agent_name]["runs_count"] += 1
    
    # Display aggregated stats
    for agent_name, stats in agent_stats.items():
        print(f"\n - {agent_name}")
        print(f"   Runs: {stats['runs_count']}")
        print(f"   Total Tokens: {stats['total_tokens']}")
        print(f"   Input Tokens: {stats['input_tokens']}")
        print(f"   Output Tokens: {stats['output_tokens']}")
        print(f"   Cost: ${stats['cost']:.6f}")
    
    print("\n" + "-" * 50)
    print(f"Overall Summary:")
    total_tokens = sum(s["total_tokens"] for s in agent_stats.values())
    total_cost = sum(s["cost"] for s in agent_stats.values())
    print(f"Total Tokens: {total_tokens}")
    print(f"Total Cost: ${total_cost:.6f}")

In [ ]:
def get_experiment_metrics(experiment_name: str) -> dict:
    """
    Get comprehensive metrics for a LangSmith experiment.
    
    Args:
        experiment_name: Name of the experiment to retrieve
    
    Returns:
        Dictionary with experiment ID, execution time, total tokens, and token breakdown by agent
    """
    langsmith_client = Client()
    
    # Get the experiment run - search by session name first
    runs = langsmith_client.list_runs(
    project_name=os.environ.get("LANGSMITH_PROJECT"),
    filter=f'eq(name, "{experiment_name}")',
    limit=1
    )
    
    run = next(iter(runs), None)
    
    if not run:
        return {"error": f"Experiment '{experiment_name}' not found"}
    
    # Calculate execution time
    execution_time = (run.end_time - run.start_time).total_seconds() if run.end_time else 0
    
    # Get all child runs
    child_runs = list(langsmith_client.list_runs(parent_run_id=run.id))
    
    # Aggregate token usage by agent name
    agent_stats = {}
    
    for agent_run in child_runs:
        agent_name = agent_run.name
        
        if agent_name not in agent_stats:
            agent_stats[agent_name] = {
                "total_tokens": 0,
                "input_tokens": 0,
                "output_tokens": 0,
                "cost": 0.0,
                "runs_count": 0
            }
        
        agent_stats[agent_name]["total_tokens"] += agent_run.total_tokens or 0
        agent_stats[agent_name]["input_tokens"] += (agent_run.input_tokens or 0)
        agent_stats[agent_name]["output_tokens"] += (agent_run.output_tokens or 0)
        agent_stats[agent_name]["cost"] += float((agent_run.completion_cost or 0.0))
        agent_stats[agent_name]["runs_count"] += 1
    
    # Build final metrics dictionary
    return {
        "run_id": str(run.id),
        "experiment_name": run.name,
        "status": run.status,
        "execution_time_seconds": execution_time,
        "total_tokens": run.total_tokens or 0,
        "total_cost": sum(s["cost"] for s in agent_stats.values()),
        "agent_stats": agent_stats
    }

In [ ]:
stats = get_experiment_metrics(experiment_name)

In [ ]:
stats